# Transfer learning

In [1]:
APPLY = True
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'sample_transfer_learning'
MODALITY = 'text'

In [2]:
from superduper import superduper, CFG

db = superduper('mongomock://test_db')

2025-Jan-13 12:53:45.83| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2025-Jan-13 12:53:45.92| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:64   | Building Data Layer
2025-Jan-13 12:53:45.92| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:79   | Data Layer built
2025-Jan-13 12:53:45.92| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:99   | Cluster initialized in 0.00 seconds.
2025-Jan-13 12:53:45.92| INFO     | Duncans-MBP.fritz.box| superduper.base.build:184  | Configuration: 
 +---------------+---------------------+
| Configuration |        Value        |
+---------------+---------------------+
|  Data Backend | mongomock://test_db |
+---------------+---------------------+


<!-- TABS -->
## Get useful sample data

In [3]:
def getter(modality='text'):
    import json
    import subprocess

    if modality == 'text': 
        subprocess.run([
            'curl', '-O', 'https://superduperdb-public-demo.s3.amazonaws.com/text_classification.json',
        ])
        with open("text_classification.json", "r") as f:
            data = json.load(f)
        subprocess.run(['rm', 'text_classification.json'])
        data = data[:200]
    else:
        subprocess.run([
            'curl', '-O', 'https://superduperdb-public-demo.s3.amazonaws.com/images_classification.zip',
        ])
        subprocess.run(['unzip', 'images_classification.zip'])
        subprocess.run(['rm', 'images_classification.zip'])
        import json
        from PIL import Image
        with open('images/images.json', 'r') as f:
            data = json.load(f)
        data = [{'x': Image.open(d['image_path']), 'y': d['label']} for d in data]
    return data

After obtaining the data, we insert it into the database.

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [4]:
if APPLY:
    data = getter()
    from superduper import Document
    ids = db[COLLECTION_NAME].insert([Document(r) for r in data]).execute()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1298k  100 1298k    0     0  1131k      0  0:00:01  0:00:01 --:--:-- 1137k


2025-Jan-13 12:53:50.81| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (table, sample_transfer_learning) not found in cache, loading from db
2025-Jan-13 12:53:50.81| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('table', 'sample_transfer_learning')) from metadata...
2025-Jan-13 12:53:50.81| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:331  | Table sample_transfer_learning does not exist, auto creating...
2025-Jan-13 12:53:51.41| WARNING  | Duncans-MBP.fritz.box| superduper.misc.annotations:296  | `superduper.ext.torch` is deprecated and will be removed in a future release. Please insteall `superduper_torch` and use `from superduper_torch import *` instead.
2025-Jan-13 12:53:51.43| WARNING  | Duncans-MBP.fritz.box| superduper.misc.annotations:296  | `superduper.ext.pillow` is deprecated and will be removed in a future release. Please insteall `superduper_pillow` and use `from superduper_pillow import *` instead.
2

<!-- TABS -->
## Compute features

In [5]:
import sentence_transformers
from superduper import Listener
from superduper_sentence_transformers import SentenceTransformer


superdupermodel_text = SentenceTransformer(
    identifier="embedding",
    model='all-MiniLM-L6-v2',
    postprocess=lambda x: x.tolist(),
)

In [6]:
import torchvision
from torchvision import transforms
from superduper_torch import TorchModel
from superduper import Listener, imported
from PIL import Image


class TorchVisionEmbedding:
    def __init__(self):
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.eval()


def preprocess(image):
    preprocess = preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    tensor_image = preprocess(image)
    return tensor_image


resnet = imported(torchvision.models.resnet18)(pretrained=True)

superdupermodel_image = TorchModel(
    identifier='my-vision-model',
    object=resnet,
    preprocess=preprocess,
    postprocess=lambda x: x.numpy().tolist()
)

/Users/dodo/.pyenv/versions/3.10.13/envs/superduper-3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dodo/.pyenv/versions/3.10.13/envs/superduper-3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
from superduper.components.model import ModelRouter


feature_extractor = ModelRouter(
    'feature_extractor',
    models={
        'text': superdupermodel_text,
        'image': superdupermodel_image,
    },
    model=MODALITY,
)

In [8]:
feature_extractor_listener = Listener(
    model=feature_extractor,
    select=db[COLLECTION_NAME].select(),
    key='x',
    identifier="features"
)


if APPLY:
    feature_extractor_listener = db.apply(
        feature_extractor_listener,
        force=True,
    )

2025-Jan-13 12:53:59.16| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1335 | Predicting with model text
2025-Jan-13 12:53:59.17| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:54:00.71| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, embedding) not found in cache, loading from db
2025-Jan-13 12:54:00.71| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('model', 'embedding')) from metadata...
2025-Jan-13 12:54:02.22| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new model:embedding:6693500c327e4484
2025-Jan-13 12:54:02.22| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, my-vision-model) not found in cache, loading from db
2025-Jan-13 12:54:02.22| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('model', 'my-vision-model')) from metadata...
2025-

200it [00:00, 41092.43it/s]

2025-Jan-13 12:54:03.90| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1345 | Predicting with model text


2025-Jan-13 12:54:05.04| INFO     | Duncans-MBP.fritz.box| superduper.components.model:672  | Adding 200 model outputs to `db`
2025-Jan-13 12:54:05.53| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:310  | Inserted 200 documents into _outputs__features__2551833d23044b81
2025-Jan-13 12:54:05.53| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.queue:120  | Consumed all events


<!-- TABS -->
## Build and train classifier

In [9]:
from superduper_sklearn import Estimator, SklearnTrainer
from sklearn.svm import SVC


scikit_model = Estimator(
    identifier="my-model-scikit",
    object=SVC(),
    trainer=SklearnTrainer(
        "my-scikit-trainer",
        key=(feature_extractor_listener.outputs, "y"),
        select=db[COLLECTION_NAME].outputs(feature_extractor_listener.predict_id),
    ),
)

In [10]:
import torch
from torch import nn
from superduper_torch.model import TorchModel
from superduper_torch.training import TorchTrainer
from torch.nn.functional import cross_entropy


class SimpleModel(nn.Module):
    def __init__(self, input_size=16, hidden_size=32, num_classes=2):
        super(SimpleModel, self).__init__()
        self.hidden_size = hidden_size
        self.fc1 = None
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        input_size = x.size(1)
        if self.fc1 is None:
            self.fc1 = nn.Linear(input_size, self.hidden_size)

        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


preprocess = lambda x: torch.tensor(x)


def postprocess(x):
    return int(x.topk(1)[1].item())


def data_transform(features, label):
    return torch.tensor(features), label


model = SimpleModel(num_classes=2)
torch_model = TorchModel(
    identifier='my-model-torch',
    object=model,
    preprocess=preprocess,
    postprocess=postprocess,
    trainer=TorchTrainer(
        key=(feature_extractor_listener.outputs, 'y'),
        identifier='my-torch-trainer',
        objective=cross_entropy,
        loader_kwargs={'batch_size': 10},
        max_iterations=1000,
        validation_interval=100,
        select=db[COLLECTION_NAME].outputs(feature_extractor_listener.predict_id),
        transform=data_transform,
    ),
)

Define a validation for evaluating the effect after training.

In [11]:
from superduper import Dataset, Metric, Validation

def acc(x, y):
    return sum([xx == yy for xx, yy in zip(x, y)]) / len(x)

accuracy = Metric(identifier="acc", object=acc)
validation = Validation(
    "transfer_learning_performance",
    key=(feature_extractor_listener.outputs, "y"),
    datasets=[
        Dataset(
            identifier="my-valid",
            select=db[COLLECTION_NAME].outputs(feature_extractor_listener.predict_id).add_fold('valid')
        )
    ],
    metrics=[accuracy],
)
scikit_model.validation = validation
torch_model.validation = validation

If we execute the apply function, then the model will be added to the database, and because the model has a Trainer, it will perform training tasks.

In [12]:
estimator = ModelRouter(
    'estimator',
    models={
        'scikit-framework': scikit_model,
        'torch-framework': torch_model,
    },
    model='scikit-framework',
)

In [13]:
if APPLY:
    db.apply(estimator, force=True)

2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (metric, acc) not found in cache, loading from db
2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('metric', 'acc')) from metadata...
2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new metric:acc:bc5a8ec93c884d82
2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (dataset, my-valid) not found in cache, loading from db
2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('dataset', 'my-valid')) from metadata...
2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new dataset:my-valid:e4b41048f9b44d47
2025-Jan-13 12:57:24.00| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (validation, transfer_learning_performance) not found in

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 189/189 [00:00<00:00, 6245.01it/s]


2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component 731e750fc6604a50 not found in cache, loading from db with uuid
2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component caac08a5bfd9490f not found in cache, loading from db with uuid
2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component bc5a8ec93c884d82 not found in cache, loading from db with uuid
2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:584  | Adding metric:acc:bc5a8ec93c884d82 to cache
2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component e4b41048f9b44d47 not found in cache, loading from db with uuid
2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:584  | Adding dataset:my-valid:e4b41048f9b44d47 to cache
2025-Jan-13 12:57:24.24| INFO     | Duncans-MBP.fritz.box| sup

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 4431.60it/s]

2025-Jan-13 12:57:24.47| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: VALID; iteration: 0; my-valid/acc: 0.5454545454545454; objective: 0.7423140406608582; 
2025-Jan-13 12:57:24.48| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 1; objective: 0.7266665697097778; 
2025-Jan-13 12:57:24.48| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 2; objective: 0.7017565965652466; 
2025-Jan-13 12:57:24.48| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 3; objective: 0.6528719663619995; 
2025-Jan-13 12:57:24.49| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 4; objective: 0.7237823605537415; 
2025-Jan-13 12:57:24.49| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 5; objective: 0.721038281917572; 
2025-Jan-13 12:57:24.49| INFO     | Duncans-MBP.fritz.box| su

...

2025-Jan-13 12:57:26.83| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 965; objective: 0.691503643989563; 
2025-Jan-13 12:57:26.84| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 966; objective: 0.6759781837463379; 
2025-Jan-13 12:57:26.84| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 967; objective: 0.6941646337509155; 
2025-Jan-13 12:57:26.84| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 968; objective: 0.6763782501220703; 
2025-Jan-13 12:57:26.84| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 969; objective: 0.7005115747451782; 
2025-Jan-13 12:57:26.85| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: TRAIN; iteration: 970; objective: 0.6830369234085083; 
2025-Jan-13 12:57:26.85| INFO     | Duncans-MBP.fritz.box| superduper_torch.trainin

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 6255.91it/s]

2025-Jan-13 12:57:26.92| INFO     | Duncans-MBP.fritz.box| superduper_torch.training:220  | fold: VALID; iteration: 1000; my-valid/acc: 0.5454545454545454; objective: 0.686010092496872; 
2025-Jan-13 12:57:26.93| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component 98d8b43746b249d0 not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.93| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component caac08a5bfd9490f not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.93| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component bc5a8ec93c884d82 not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.93| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:584  | Adding metric:acc:bc5a8ec93c884d82 to cache
2025-Jan-13 12:57:26.93| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component e4b41048f9b44d47 not found in cache, loading from db with uuid
2025-Jan-13 12:57


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 6101.21it/s]

2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component 98d8b43746b249d0 not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component caac08a5bfd9490f not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component bc5a8ec93c884d82 not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:584  | Adding metric:acc:bc5a8ec93c884d82 to cache
2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:562  | Component e4b41048f9b44d47 not found in cache, loading from db with uuid
2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:584  | Adding dataset:my-valid:e4b41048f9b44d47 to cache
2025-Jan-13 12:57:26.97| INFO     | Duncans-MBP.fritz.box| sup

Get the training metrics

In [14]:
if APPLY:
    model = db.load('model', 'my-model-scikit')
    model.metric_values

In [ ]:
from superduper import Application

application = Application(
    identifier='transfer-learning',
    components=[feature_extractor_listener, estimator],
)

In [ ]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData

t = Template(
    'transfer_learning',
    default_table=Table(
        'sample_transfer_learning',
        schema=Schema(
            'sample_transfer_learning/schema',
            fields={'x': 'str', 'y': 'int'},
        ),
        data=RemoteData(
            'text_classification',
            getter=getter,
        ),
    ),
    template=application,
    substitutions={'docs': 'table_name', 'text': 'modality'},
    template_variables=['table_name', 'framework', 'modality'],
    types={
        'table_name': {
            'type': 'str',
            'default': 'sample_transfer_learning',
        },
        'modality': {
            'type': 'str',
            'default': 'text',
        },
        'framework': {
            'type': 'str',
            'default': 'scikit-framework',
        },
    }
)

In [ ]:
t.export('.')